## BIOENG-310: Neuroscience Foundations for Engineers

Notebook created by Martin Schrimpf, edited by Alejandro Rodriguez Guajardo and Yingtian Tang.

# Week 5: Data visualization and linear probing for V1 neural activities

This week, we **visualize neural firing data** from the primary visual cortex (V1) of a macaque [(Freeman & Ziemba, 2013)](https://www.nature.com/articles/nn.3402). V1 is known to encode low-level features of visual stimuli, such as texture. To assess this, we apply a technique known as **linear readout** or **linear probing**. This method involves training a simple linear classifier, such as logistic regression or a support vector machine, on neural activity to predict stimulus features.

The exercise is written in this *jupyter notebook*, which is a notebook of interleaved [Markdown](https://jupyter-notebook.readthedocs.io/en/latest/notebook.html#markdown-cells) and [Python blocks](https://jupyter-notebook.readthedocs.io/en/latest/notebook.html#code-cells). All of the blocks can be executed: Markdown blocks turn into text (so when they look strange, just execute them); Python blocks execute the code while keeping the variables so they can be used in other Python blocks. You can check more information on the basics of jupyter notebook [starting from this section of the whole webpage](https://jupyter-notebook.readthedocs.io/en/latest/notebook.html#notebook-user-interface).

The notebook consists of the following sessions:
- **Load Data**: load the experimental V1 neural data
- **Stimuli**: load the stimuli used for the recording experiment
- **Data visualization**: visualize the timeseries of V1 firing patterns (lots of work)
- **Linear readout / linear probing**: use V1 activities to predict the texture of stimuli (lots of work)

Please complete the code in the sections marked '''%%...%%''' based on the given instructions and context. Then, run all the blocks to see the results.
Additional hints:
- You can open some temporary Python blocks as your scratchpad.
- Be careful of the ordering of running Python blocks, since the variables are shared.
- References are important: look at the input/output specifications of the functions.

Enjoy the journey! 🚀

**Before you begin**, make sure you have all dependencies installed (see `requirements.txt` in this week's folder). You can install them by executing the following block. (The following code might require you to *restart runtime*, which is fine)

In [ ]:
!pip install jupyter brainscore-vision matplotlib

### Load Data

At the current state, it can unfortunately often be difficult to access brain recordings. Not all groups share their data (although new regulations are improving this), and even if the data is accessible it is often difficult to interact with due to non-standard file formats, missing/unclear metadata, and obscure processing pipelines.

That being said, we will here save you the headache by starting from a packaged and standardized dataset.
This data was first published by [Freeman* & Ziemba* et al. 2013](https://www.nature.com/articles/nn.3402) and subsequently packaged into the **[Brain-Score](www.brain-score.org)** platform.
The format of the data here is in [xarray](https://xarray.dev), a structure allowing for multi-dimensional data with multiple metadata along all dimensions. If you know pandas, xarray is the multi-dimensional extension of it. Check basics of xarray [here](https://tutorial.xarray.dev/overview/xarray-in-45-min.html) and pandas [here](https://www.w3schools.com/python/pandas/default.asp).

In [ ]:
import brainscore_vision

# brainscore will download the data for you
data = brainscore_vision.load_dataset('FreemanZiemba2013.public')

# we'll focus on only V1 recordings in this exercise
v1_data = data.sel(region='V1')

# By just typing the name of a variable, jupyter will show its content.
# In this case, the xarray 'v1_data' will be presented with (scroll in the below to have a full view; toggle on the left to show/hide them):
#   1. basics of this data structure: bytes, shape, data type, etc.
#   2. a graphical representation of its different dimensions
#   3. a section called 'coordinates' that shows the names of the dimensions and their associated values
v1_data

Let's get a basic understanding of the neural recording by examining the *coordinates*. Answer the following questions:

How many presentation trials?

How many neural sites?

The temporal resolution of the recording is 1 ms per time bin. Then, how long is each trial?

Each *neural site* is recorded using a single quartz-platinum-tungsten microelectrode implanted in the macaque brain. Do you think each neural site records the activity of a single neuron, or could it capture signals from multiple neurons? Explain your reasoning.

### Stimuli

How did this data come about in the first place? Primate subjects were presented with images while experimenters were recording from early visual cortex.

What are those images?

In [ ]:
%matplotlib inline
from matplotlib import pyplot, image

# The xarray 'v1_data' has 3 attributes (accesse all of them by 'v1_data.attrs')
#   1. stimulus_set_identifier: name of the stimulus set
#   2. stimulus_set: a brainscore StimulusSet (a subclass of pandas.Dataframe) that records all the information about the stimulus set
#   3. identifier: name of the whole experiment
stimuli = v1_data.stimulus_set

# a shortcut to access the *first* stimulus is in the stimulus set
single_stimulus_id = stimuli['stimulus_id'].values[0]
# note that the 'stimulus_id's are also present in the coordinates of the xarray 'v1_data'.
# however, generally those ones are in arbitrary order and the same stimlus_id can be repeated.
# therefore, it is better to access the 'stimulus_id' from the stimulus set.

# use a method 'get_stimulus' of StimulusSet to get the stimulus storage path given the id
image_path = stimuli.get_stimulus(single_stimulus_id)

# show the image using matplotlib
image_content = image.imread(image_path)
pyplot.imshow(image_content, cmap='gray')
pyplot.show()

In [ ]:
# Let's look at some more stimuli.
fig, axes = pyplot.subplots(nrows=1, ncols=10, figsize=(30, 3))

for i, ax in enumerate(axes.flatten()):
    
    # loop through multiple stimuli, retrieve their IDs, get their image paths, load the images, and display them in grayscale
    # we now focus on the i-th stimulus 
    image_path = """%% Retrive the image path of the i-th stimulus %%"""
    image_content = image.imread(image_path)
    ax.imshow(image_content, cmap='gray')
    ax.set_axis_off()
    
pyplot.show()

### Data visualization

Now that we've seen the images shown to the macaques, let's check the neural activity recorded in V1. It's always a good idea to first plot the raw activity data before running any complex analysis. This helps ensure the data looks as expected and there are no obvious issues.

Let's start with the response to the single stimulus that we visualized in the previous section.

In [ ]:
single_stimulus_id = '''%% Again, retrieve the first stimulus ID from the dataset. %%'''

# xarray.sel method allows slicing the currency xarray along one/multiple coordinate(s) or dimension(s)
# reference: https://docs.xarray.dev/en/latest/generated/xarray.DataArray.sel.html
# hint: you can select along either coordinate(s) or dimension(s)
stimulus_data = v1_data.sel('''%% Select the data for the chosen stimulus. %%''')

# before executing, think about the shape/coordinates of this xarray
stimulus_data

We see that this single stimulus was shown a total of 20 times (repetitions), has 205 neural sites recorded, and was collected over 300 time-bins (in this case 1 ms each).
In visual neuroscience, repeated trials are usually conducted to average out noise. Let's do just that by averaging all the repetitions.

In [ ]:
from brainscore_vision.benchmark_helpers.neural_common import average_repetition

# average over repeated repetitions of the same stimulus using the brainscore function
averaged_stimulus_data = average_repetition(stimulus_data)

# remove (or 'squeeze') the singleton dimension after the averaging along it
averaged_stimulus_data = averaged_stimulus_data.squeeze('presentation')

# before executing, think about the shape/coordinates of this xarray
averaged_stimulus_data

Now we visualize the averaged recordings for this specific stimulus:

In [ ]:
from mpl_toolkits.axes_grid1 import make_axes_locatable

fig, ax = pyplot.subplots()

# Construct a colormap that visualize the spiking activity of the neuron
colormap = ax.imshow(averaged_stimulus_data.transpose('neuroid_id', 'time_bin').values, cmap='gray')

def add_colorbar(fig, ax, colormap):
    divider = make_axes_locatable(ax)  # make space for colorbar
    ax_colorbar = divider.append_axes('right', size='5%', pad=0.05)
    colorbar = fig.colorbar(colormap, cax=ax_colorbar)
    colorbar.set_label('''%% Provide a label describing what the color represents %%''')

add_colorbar(fig, ax, colormap)

# %% Label the axes %%
ax.set_xlabel('''%% Label the x-axis appropriately %%''')
ax.set_ylabel('''%% Label the y-axis appropriately %%''')

pyplot.show()

Cool! Lots of activity. But always think of what the 'time' and 'neural site' dimension mean. This will be helpful.

This was for one single stimulus, let's see what the data looks like on average across all the stimuli.

In [ ]:
from mpl_toolkits.axes_grid1 import make_axes_locatable

fig, ax = pyplot.subplots()

# average over all stimuli, all repetitions
# reference: https://docs.xarray.dev/en/stable/generated/xarray.DataArray.mean.html
averaged_v1_data = v1_data.mean('''%% Specify the correct dimension to average over %%''')

colormap = ax.imshow(averaged_v1_data.transpose('''%% Fill in the dimensions in the order you prefer %%''').values, cmap='gray')
add_colorbar(fig, ax, colormap)
ax.set_xlabel('''%% Label the x-axis appropriately %%''')
ax.set_ylabel('''%% Label the y-axis appropriately %%''')
pyplot.show()

There is another way to show these activity patterns: plotting them as timeseries.

In [ ]:
averaged_v1_data.load()  # make sure all data is loaded (sometimes the xarray is lazily loaded)

fig, ax = pyplot.subplots()
time_bins = averaged_v1_data['time_bin_start'].values

# plot individual sites
neural_sites = averaged_v1_data['neuroid_id'].values
for neural_site in neural_sites:

    # here we demonstrate an example of multi-slicing in xarray, even if in this case it is not necessary.
    # xarray.sel does not directly support slicing with multiple values, but this can be achieved in two ways:
    match_site = [neuroid_id == neural_site for neuroid_id in neural_sites]  # a binary mask
    match_idx = [i for i, neuroid_id in enumerate(neural_sites) if neuroid_id == neural_site]  # a list of indices
    
    # the *first* way:
    site_data = averaged_v1_data[{'neuroid_id': match_site}]
    # or
    site_data = averaged_v1_data[{'neuroid_id': match_idx}]

    # the *second* way:
    site_data = averaged_v1_data.isel(neuroid_id=match_site)
    # or
    site_data = averaged_v1_data.isel(neuroid_id=match_idx)

    # more information on xarray slicing at: https://docs.xarray.dev/en/latest/user-guide/indexing.html#vectorized-indexing

    site_data = site_data.squeeze()
    ax.plot(time_bins, site_data.values, color='gray', alpha=0.3)

# also plot site average
site_average = averaged_v1_data.mean('''%% Compute the average firing rate time series across all sites. %%''')
ax.plot(time_bins, site_average.values, color='darkgray')

# and the sites that are more active -- let's say the sites that at some point are active above 0.05
active_sites = (averaged_v1_data > 0.05).any('time_bin')
active_sites = averaged_v1_data[{'neuroid_id': active_sites}].mean('neuroid_id')
ax.plot(time_bins, active_sites.values, color='black')

ax.set_xlabel('Time (ms)')
ax.set_ylabel('Normalized firing rate')
pyplot.show()

What happens at time=0ms?

When are most neural sites start to become active?

Why does it take time for sites to become active?

You guess: why is there activity again at around 270ms?

You guess: why do some sites show small activity (e.g. towards the bottom)?

#### Bonus: Reproducing a figure from the paper

In the original [paper](https://www.nature.com/articles/nn.3402) we took the data from, Figure 2b identifies a contrast of neural responses to noise and naturalistic stimuli between neural sites in V1 and V2.
In other words, the neural activity shows different patterns across different stimulus types: texture *vs.* noise.

With our data processing so far, we can reproduce this figure.

In [ ]:
from brainio.assemblies import NeuronRecordingAssembly

full_data = brainscore_vision.load_dataset('FreemanZiemba2013.public')  # make sure we have both the V1 and V2 data again

def process_data(data: NeuronRecordingAssembly) -> tuple[NeuronRecordingAssembly, NeuronRecordingAssembly]:
    data = data.mean('neuroid_id')
    # note that in the paper these values are further normalized by each site's maximum firing rate
    data.load()  # make sure data is loaded
    texture_data = data.sel(texture_type='texture').mean('presentation')
    noise_data = data.sel(texture_type='noise').mean('presentation')
    return texture_data, noise_data

v1_data = '''%% Extract only the regional data of V1 using .sel() %%'''
v2_data = '''%% Extract only the regional data of V2 using .sel() %%'''

fig, axes = pyplot.subplots(nrows=2)

for title, ax, data, base_color in zip(
        ['V1', 'V2'], axes, [v1_data, v2_data], ['green', 'blue']):
    texture_data, noise_data = process_data(data)
    time_bins = data['time_bin_start'].values
    ax.plot(time_bins, texture_data.values, color=f"dark{base_color}", label='texture')
    ax.plot(time_bins, noise_data.values, color=f"light{base_color}", label='noise')
    ax.legend()
    ax.set_title(title)
    ax.set_ylabel('Normalized firing rate')
    if title == 'V2':
        ax.set_xlabel('Time (ms)')
    else:
        ax.xaxis.set_ticklabels([])  # hide tick labels on top plot
pyplot.show()

What do you see? Compare yours to the original figures.

### Linear readout / linear probing

Since neural firing patterns vary with different stimuli, we want to understand what information these patterns truly encode about the stimuli.

A very common technique to investigate the informational content of data is to see if a linear regression can predict some variables of interests.

In our case, we're interested in whether V1 activity can predict if the stimuli contain texture or if they are simply pure noise.

#### Time-averaging

To make things simpler, we will average out the time dimension from now on.

In other words, we assume that the time-averaged activity magnitude, without considering dynamics, already encodes texture information.

As we observed before, most of the interesting signal is localized to 50-200ms. Let's average out the time dimension in that range so that we can focus on this signal. We also again average over the repetitions of each stimulus.

In [ ]:
from brainscore_vision.benchmark_helpers.neural_common import average_repetition

def average_time_range(data, time_window):
    data = '''%% Select only the time bins within the given time_window %%'''
    data = '''%% Compute the mean over the time dimension %%'''
    # note that you can pass keep_attrs=True to keep the 'attrs' of the xarray for some xarray operations
    return data

repetition_average = average_repetition(v1_data)
time_average = average_time_range(repetition_average, time_window = (50, 200))
time_average

Now without the time dimension, we're left with the responses 102 V1 neural sites to 135 stimuli.

We can visualize this again using an 'imshow' plot, but please note that the axes have changed compared to the previous plots.

In [ ]:
fig, ax = pyplot.subplots()
colormap = ax.imshow(time_average.values, cmap='Greens')
add_colorbar(fig, ax, colormap)
ax.set_xlabel('Stimuli (image #)')
ax.set_ylabel('Neural site (#)')
pyplot.show()

Arguably it's not super obvious how much signal there is that is specific to images. We will analyze this in detail in the next few steps.

#### Predicting/decoding the texture types

For this dataset, there are two types of stimuli: 'noise' images and 'texture' images. Let's see if we can differentiate among them using the V1 neural data.

Concretely, we fit a classifier that predicts the texture type based on the neural firing rates.
We want to see if we can really *predict* unseen data. Therefore, we split our data into a `train` and a `validation` split. We train the classifier only on the `train` split. Then, we check if the trained classifier has high accuracy on the `validation` split.

The scikit-learn package has some great utilities for such simple classic machine learning tools.

Note: In research, we would also use an additional `test` split that we hold out altogether and only analyze at the very end when all model parameters are locked in, to avoid implicit overfitting to the data.


In [ ]:
from sklearn.model_selection import train_test_split

# a shortcut for getting all the stimulus ids as a numpy array
stimulus_ids = time_average['stimulus_id'].values

# use 80% for training, 20% for validation
# reference: https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html
train_stimuli, val_stimuli = '''%% use train_test_split to divide the stimulus_ids into 80% training and 20% validation, use random_state=3 %%'''

# hint: for the following, use the multi-slicing technique we demonstrated earlier
train_data = '''%% Write the code to select the training data using train_stimuli here %%'''
val_data = '''%% Write the code to select the validation data using val_stimuli here %%'''

# %% Reminder: After this, you should have train_data and val_data that are ready to be used for further analysis or model training.

What's chance accuracy on this dataset?

In [ ]:
from numpy.random import RandomState

train_y = train_data['texture_type'].values
random_baseline = RandomState(seed=42).choice(train_y, replace=False, size=len(train_y))
chance_accuracy = (random_baseline == train_y).mean()
print("chance performance is", chance_accuracy)

With `random_state=3` for `train_test_split`, the chance is actually exactly 50% in this specific case; depending on randomness one run of this might give values slightly below/above 0.5.  

Now we train a linear regression from the train brain data to the train stimulus types.

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler

# Reference: https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html
linear_readout = LogisticRegression()

# reshape the training data appropriately by transposing the 'presentation' and 'neuroid_id' axes.
# then, use .values to access the numpy array
train_x = '''%% Write the code to extract the training features (train_x) here %%'''
train_y = '''%% Write the code to extract the target labels (train_y) here %%'''

# normalize data
scaler = StandardScaler().fit(train_x)
train_x = scaler.transform(train_x)

# fit the decoder
linear_readout.fit(train_x, train_y)

# double-checking that the fitting worked
train_predictions = linear_readout.predict(train_x)
train_accuracy = (train_predictions == train_y).mean()
print("train accuracy is", train_accuracy)

Great, we were able to (over)fit the training data. How well does this work on the validation data?

In [ ]:
val_x = '''%% Write the code to extract the validation features (val_x) here %%'''
val_y = '''%% Write the code to extract the target labels (val_y) here %%'''

# don't forget to normalize the inputs again
val_x = scaler.transform(val_x)

# evaluate the decoder on the validation set
val_accuracy = linear_readout.score(val_x, val_y)
print("validation accuracy is", val_accuracy)

Not so bad.

Why did we not get 100% validation accuracy?

How could you improve the accuracy of the classifier?

Taking the [paper](https://www.nature.com/articles/nn.3402)'s Figure 2 into account, do you think the classifier performance with V2 data would be different than with V1?